## Encoder Decoder

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_parquet("../0 - Data/4 - scaled/ft_strategy_1_scaled.pq")
df.head(2)

In [ ]:
X = df.drop(columns=['Is Fraud'])

# Set the dimensions for the encoder and bottleneck (latent) layer
input_dim = X.shape[1] 
encoding_dim = 50 

X.head(2)

1. Define the Autoencoder Architecture

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam


# Encoder
input_layer = Input(shape=(input_dim,))
encoder = Dense(64, activation='relu')(input_layer)
encoder = Dense(32, activation='relu')(encoder)
bottleneck = Dense(encoding_dim, activation='relu')(encoder)  # Bottleneck layer

# Decoder
decoder = Dense(32, activation='relu')(bottleneck)
decoder = Dense(64, activation='relu')(decoder)
output_layer = Dense(input_dim, activation='linear')(decoder)  # Reconstruction output

# Build the Autoencoder Model
autoencoder = Model(inputs=input_layer, outputs=output_layer)

# Compile the Model
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

2. Train the Auto-Encoder

In [ ]:
autoencoder.fit(X, X, epochs=20, batch_size=64, validation_split=0.2, verbose=1)

3. Extract the Encoder Model for Dimensionality Reduction

In [ ]:
encoder_model = Model(inputs=input_layer, outputs=bottleneck)

# Transform the data into the reduced dimension using the encoder
X_reduced = encoder_model.predict(X)

# Convert the reduced data into a DataFrame and add the target column
reduced_df = pd.DataFrame(X_reduced, columns=[f'Encoded_{i+1}' for i in range(encoding_dim)])
reduced_df['Is Fraud'] = df['Is Fraud'].values

4. Save the reduced DataFrame to a file

In [ ]:
reduced_df.to_parquet("../0 - Data/5 - pcas/ft_strategy_1_ae_reduction.pq")